In [1]:
import numpy as np
import pandas as pd
# from process import get_data

In [2]:
!wget https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/refs/heads/master/ann_logistic_extra/ecommerce_data.csv

--2025-05-10 11:23:47--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/refs/heads/master/ann_logistic_extra/ecommerce_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12381 (12K) [text/plain]
Saving to: ‘ecommerce_data.csv’

ecommerce_data.csv  100%[===================>]  12.09K  --.-KB/s    in 0s      

2025-05-10 11:23:47 (28.1 MB/s) - ‘ecommerce_data.csv’ saved [12381/12381]



In [ ]:
def get_data():
    df = pd.read_csv('ecommerce_data.csv')
    # df.head()
    # easier to work with numpy array
    data = df.to_numpy()

    # shuffle our data, by the row. The columns don't get mixed
    np.random.shuffle(data)

    # split features and labels
    X = data[:, :-1]
    Y = data[:, -1].astype(np.int32)

    # one-hot encoding for time of day
    N, D = X.shape
    X2 = np.zeros((N, D+3))  # add only three because we can replace one column
    X2[:, :(D-1)] = X[:, :(D-1)]

    for n in range(N):
        t = int(X[n, D-1])  # t is the value in that cell
        X2[n, t+D-1] = 1    # puts a value of 1 in the t-th column of that row
                            # adds 1 in col. 7 for a value 3, adds 1 in col. 6

    # method 2 (more efficient)
    # Z = np.zeros((N, 4))
    # Z[np.arange(N), X[:, D-1].astype(np.int32)] = 1
    # Z[(r1, r2, r3, ...), (c1, c2, c3, ...)] = value
    # X2[:, -4:] = Z

    # assign X2 back to X
    X = X2

    # split train and test
    Xtrain = X[:-100]
    Ytrain = Y[:-100]
    Xtest = X[-100:]
    Ytest = Y[-100:]

    # normalize columns 1 and 2
    for i in (1, 2):
        m = Xtrain[:, i].mean()
        s = Xtrain[:, i].std()
        Xtrain[:, i] = (Xtrain[:, i] - m) / s
        Xtest[:, i] = (Xtest[:, i] - m) / s

    return Xtrain, Ytrain, Xtest, Ytest

In [ ]:
X, Y, _, _ = get_data()
np.shape(X)
np.shape(Y)

(400,)

In [ ]:
# randomly initialized weights
M = 5
D = X.shape[1]
K = len(set(Y))

W1 = np.random.randn(D, M)
b1 = np.zeros(M)
W2 = np.random.randn(M, K)
b2 = np.zeros(K)

In [ ]:
# functions to compute the output
def softmax(a):
    expa = np.exp(a)
    return expa / expa.sum(axis=1, keepdims=True)

def forward(X, W1, b1, W2, b2):
    Z = np.tanh(X.dot(W1) + b1)  # hidden layer
    return softmax(Z.dot(W2) + b2)

In [ ]:
P_Y_given_X = forward(X, W1, b1, W2, b2)
P_Y_given_X.shape

(400, 4)

In [ ]:
predictions = np.argmax(P_Y_given_X, axis=1)

In [ ]:
predictions

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 3, 0, 0, 1,
       0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 2, 2, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 3,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 3, 0, 1, 1, 0, 2, 2, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 1, 0, 0, 0, 2, 2, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 3, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       2, 1, 0, 1, 0, 0, 3, 1, 1, 0, 1, 0, 0, 0, 1,

In [ ]:
def classification_rate(Y, P):
    return np.mean(Y==P)

In [ ]:
print(f"Score: {classification_rate(Y, predictions)}")

Score: 0.45
